This notebook is a utility to move and organize files for the block based synchronization paradigm - it is a pre-requisite for running the pipeline

In [5]:
import pathlib
import os
import shutil
import datetime
import subprocess as sp
import re
import pandas as pd
import numpy as np
import tqdm
import glob
from lxml import etree
from BlockSync2 import *
import scipy.io

In [8]:
import json
import csv
import pathlib
import subprocess as sp
import numpy as np

# Function to read the framerate from the JSON file
def get_framerate_from_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
        # Use the frame_duration_us to compute framerate
        frame_duration_us = data.get("frame_duration_us", 16666)  # Default to 16666 us if not found
        framerate = 1_000_000 / frame_duration_us  # Convert microseconds to seconds
        return framerate

# Function to calculate the FPS from the timestamps file
def calculate_fps_from_timestamps(timestamps_path):
    timestamps = []

    # Read the timestamps from the CSV file
    with open(timestamps_path, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header
        for row in reader:
            timestamps.append(float(row[0]))  # Assuming the first column contains the timestamps

    # Calculate the time differences (frame durations) between consecutive timestamps
    frame_durations = np.diff(timestamps)  # Time difference between consecutive frames

    # Calculate the average frame duration and convert it to FPS
    avg_frame_duration = np.mean(frame_durations)
    fps = 1 / avg_frame_duration if avg_frame_duration > 0 else 30  # Default to 30 if something goes wrong

    return fps

# Function to convert .h264 to .mp4 with the actual framerate
def convert_to_mp4_with_comparison(h264_path, json_path, timestamps_path, output_path):
    # Get framerate from the .json file
    json_fps = get_framerate_from_json(json_path)
    print(f'Framerate from JSON: {json_fps:.2f}')

    # Calculate framerate from the timestamps file
    csv_fps = calculate_fps_from_timestamps(timestamps_path)
    print(f'Calculated FPS from timestamps: {csv_fps:.2f}')

    # Choose the higher of the two framerates (or the one with the most accurate timestamps)
    fps = max(json_fps, csv_fps)
    print(f'Using {fps:.2f} FPS for conversion')

    # Convert to .mp4 using MP4Box, setting the fps based on the calculated value
    try:
        sp.run(['MP4Box', '-fps', str(fps), '-add', str(h264_path), str(output_path)], check=True)
        print(f'Conversion to {output_path} completed with {fps:.2f} FPS.')
    except sp.CalledProcessError as e:
        print(f'Error during conversion: {e}')

# Function to automate the conversion process for all folders in the directory
def convert_all_folders_in_tree(base_folder_path):
    base_folder_path = pathlib.Path(base_folder_path)

    # Walk through each folder in the base folder
    for folder in base_folder_path.rglob('*'):
        if folder.is_dir():
            h264_file = list(folder.glob('*.h264'))
            json_file = list(folder.glob('*.json'))
            timestamps_file = list(folder.glob('*_timestamps.csv'))

            # Check if the necessary files exist
            if h264_file and json_file and timestamps_file:
                h264_path = h264_file[0]
                json_path = json_file[0]
                timestamps_path = timestamps_file[0]

                # Generate the output mp4 file path
                output_file = h264_path.with_suffix('.mp4')

                # Convert the .h264 file to .mp4 with the correct framerate
                convert_to_mp4_with_comparison(h264_path, json_path, timestamps_path, output_file)
            else:
                print(f"Skipping folder {folder}: Missing required files")

# Example Usage: Convert all folders in the given directory
base_folder_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_208\2025_12_09')  # Base path to start the conversion process
convert_all_folders_in_tree(base_folder_path)


Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_013: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_014: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_015: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_016: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_017: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_018: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_013\analysis: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_013\arena_videos: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_013\eye_videos: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\block_013\oe_files: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_09\bl

In [9]:
# Example Usage: Convert all folders in the given directory
base_folder_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_208\2025_12_14')  # Base path to start the conversion process
convert_all_folders_in_tree(base_folder_path)

Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_019: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_020: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_021: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_019\analysis: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_019\arena_videos: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_019\eye_videos: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_019\oe_files: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_019\arena_videos\trials_images: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_019\arena_videos\videos: Missing required files
Skipping folder Z:\Nimrod\experiments\PV_208\2025_12_14\block_019\arena_videos\videos\frames_timestam

In [4]:
# collect AVI files for block conversion in preytouch system:
def find_avi_files(base_directory):
    avi_files = []
    for root, _, files in os.walk(base_directory):
        for file in files:
            if file.lower().endswith('.avi'):  # Case-insensitive check for .avi
                avi_files.append(os.path.join(root, file))
    return avi_files
base_dir = pathlib.Path(r'Z:\Nimrod\experiments\PV_208\Unsorted\PreyTouch-208\20251202')
avi_files = find_avi_files(base_directory=base_dir)


In [5]:
avi_files

['Z:\\Nimrod\\experiments\\PV_208\\Unsorted\\PreyTouch-208\\20251202\\block1\\videos\\front_20251202T150425.avi',
 'Z:\\Nimrod\\experiments\\PV_208\\Unsorted\\PreyTouch-208\\20251202\\block1\\videos\\left_20251202T150425.avi',
 'Z:\\Nimrod\\experiments\\PV_208\\Unsorted\\PreyTouch-208\\20251202\\block1\\videos\\right_20251202T150425.avi',
 'Z:\\Nimrod\\experiments\\PV_208\\Unsorted\\PreyTouch-208\\20251202\\block1\\videos\\top_20251202T150425.avi']

In [6]:
# Convert AVI -> MP4, preserving source timing (no fixed -r), robust to MJPEG-in-AVI
import os
import json
import pathlib
import subprocess
from fractions import Fraction

def _ffprobe_stream_info(path):
    cmd = [
        "ffprobe", "-v", "error",
        "-print_format", "json",
        "-show_streams",
        "-select_streams", "v:0",
        str(path)
    ]
    out = subprocess.check_output(cmd)
    data = json.loads(out.decode("utf-8"))
    streams = data.get("streams", [])
    return streams[0] if streams else None

def _fraction_to_float(frac_str):
    try:
        return float(Fraction(frac_str))
    except Exception:
        return None

def convert_and_organize_avi_files(avi_file_paths, crf=18, preset="slow", overwrite=False):
    for avi_path in map(pathlib.Path, avi_file_paths):
        try:
            if "avi_version" in avi_path.parts:
                print(f"Skipping {avi_path} (already in 'avi_version').")
                continue
            if not avi_path.exists():
                print(f"Missing: {avi_path}")
                continue

            base_dir = avi_path.parent
            base_name = avi_path.stem
            mp4_path = base_dir / f"{base_name}.mp4"
            avi_version_dir = base_dir / "avi_version"
            avi_version_dir.mkdir(exist_ok=True)

            vstream = _ffprobe_stream_info(avi_path)
            if vstream:
                r_fps = _fraction_to_float(vstream.get("r_frame_rate", "0/1"))
                avg_fps = _fraction_to_float(vstream.get("avg_frame_rate", "0/1"))
                time_base = vstream.get("time_base")
                codec_name = vstream.get("codec_name")
                pix_fmt_in = vstream.get("pix_fmt")
                print(f"[{avi_path.name}] codec={codec_name}, r_fps={r_fps}, avg_fps={avg_fps}, time_base={time_base}, pix_fmt={pix_fmt_in}")
            else:
                print(f"[{avi_path.name}] Could not read video stream info; proceeding.")

            if mp4_path.exists() and not overwrite:
                print(f"MP4 already exists, skipping encode: {mp4_path}")
            else:
                ffmpeg_command = [
                    "ffmpeg",
                    "-hide_banner",
                    "-loglevel", "info",    # show useful errors
                    "-y" if overwrite else "-n",
                    "-fflags", "+genpts",   # synthesize PTS if missing
                    "-i", str(avi_path),

                    # map video and optional audio (the ? makes it optional)
                    "-map", "0:v:0",
                    "-map", "0:a?",

                    # video encode
                    "-c:v", "libx264",
                    "-preset", preset,
                    "-crf", str(crf),
                    "-pix_fmt", "yuv420p",  # safe for MP4/x264

                    # keep original timing (variable timestamps ok)
                    "-vsync", "vfr",

                    # audio encode only if audio present
                    "-c:a", "aac",
                    "-b:a", "192k",

                    # mp4 faststart for web playback
                    "-movflags", "+faststart",

                    str(mp4_path)
                ]

                print(f"Converting {avi_path} -> {mp4_path} (preserving timestamps/VFR)…")
                # Capture stdout+stderr to print on failure
                proc = subprocess.run(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                if proc.returncode != 0:
                    print(proc.stdout)
                    print(proc.stderr)
                    raise subprocess.CalledProcessError(proc.returncode, ffmpeg_command, output=proc.stdout, stderr=proc.stderr)

            # Move original .avi into avi_version/
            moved_avi_path = avi_version_dir / avi_path.name
            if moved_avi_path.exists():
                i = 1
                while (avi_version_dir / f"{avi_path.stem}__orig{i}.avi").exists():
                    i += 1
                moved_avi_path = avi_version_dir / f"{avi_path.stem}__orig{i}.avi"

            avi_path.rename(moved_avi_path)
            print(f"Moved original AVI to {moved_avi_path}")

        except subprocess.CalledProcessError as e:
            print(f"ffmpeg error for {avi_path}:\n{e.stderr or e.output or e}")
        except Exception as e:
            print(f"Error processing {avi_path}: {e}")


# Example:
convert_and_organize_avi_files(avi_files, crf=18, preset="slow")


[front_20251202T150425.avi] codec=mjpeg, r_fps=58.824, avg_fps=58.824, time_base=125/7353, pix_fmt=yuvj420p
Converting Z:\Nimrod\experiments\PV_208\Unsorted\PreyTouch-208\20251202\block1\videos\front_20251202T150425.avi -> Z:\Nimrod\experiments\PV_208\Unsorted\PreyTouch-208\20251202\block1\videos\front_20251202T150425.mp4 (preserving timestamps/VFR)…
Moved original AVI to Z:\Nimrod\experiments\PV_208\Unsorted\PreyTouch-208\20251202\block1\videos\avi_version\front_20251202T150425.avi
[left_20251202T150425.avi] codec=mjpeg, r_fps=58.824, avg_fps=58.824, time_base=125/7353, pix_fmt=yuvj420p
Converting Z:\Nimrod\experiments\PV_208\Unsorted\PreyTouch-208\20251202\block1\videos\left_20251202T150425.avi -> Z:\Nimrod\experiments\PV_208\Unsorted\PreyTouch-208\20251202\block1\videos\left_20251202T150425.mp4 (preserving timestamps/VFR)…
Moved original AVI to Z:\Nimrod\experiments\PV_208\Unsorted\PreyTouch-208\20251202\block1\videos\avi_version\left_20251202T150425.avi
[right_20251202T150425.avi] 

In [5]:
# This function takes in a list of avi files and translates them to mp4 while keeping them at a subfolder for later use (no deletes)
import os
import subprocess

def convert_and_organize_avi_files(avi_file_paths):
    for avi_path in avi_file_paths:
        try:
            # Skip files already in the 'avi_version' folder
            if "avi_version" in pathlib.Path(avi_path).parts:
                print(f"Skipping {avi_path} as it is already in the 'avi_version' folder.")
                continue

            # Determine the target frame rate
            frame_rate = 30 if 'left' in os.path.basename(avi_path).lower() else 60

            # Determine output paths
            base_dir = os.path.dirname(avi_path)
            base_name = os.path.splitext(os.path.basename(avi_path))[0]
            mp4_path = os.path.join(base_dir, f"{base_name}.mp4")
            avi_version_dir = os.path.join(base_dir, "avi_version")

            # Ensure the 'avi_version' folder exists
            os.makedirs(avi_version_dir, exist_ok=True)

            # Build and run the ffmpeg command
            ffmpeg_command = [
                "ffmpeg",
                "-i", avi_path,             # Input file
                "-r", str(frame_rate),      # Frame rate
                "-vcodec", "libx264",       # Video codec
                "-acodec", "aac",           # Audio codec
                mp4_path                    # Output file
            ]
            print(f"Converting {avi_path} to {mp4_path} at {frame_rate} fps...")
            subprocess.run(ffmpeg_command, check=True)

            # Move the original .avi file to the 'avi_version' folder
            moved_avi_path = os.path.join(avi_version_dir, os.path.basename(avi_path))
            os.rename(avi_path, moved_avi_path)
            print(f"Moved original .avi file to {moved_avi_path}")

        except subprocess.CalledProcessError as e:
            print(f"Error converting {avi_path}: {e}")
        except Exception as e:
            print(f"Error processing {avi_path}: {e}")

# Example usage:

convert_and_organize_avi_files(avi_files)


Converting X:\Nimrod\pv_106_unsorted_data_04_09_25\block4\videos\front_20250904T132458.avi to X:\Nimrod\pv_106_unsorted_data_04_09_25\block4\videos\front_20250904T132458.mp4 at 60 fps...
Moved original .avi file to X:\Nimrod\pv_106_unsorted_data_04_09_25\block4\videos\avi_version\front_20250904T132458.avi
Converting X:\Nimrod\pv_106_unsorted_data_04_09_25\block4\videos\left_20250904T132458.avi to X:\Nimrod\pv_106_unsorted_data_04_09_25\block4\videos\left_20250904T132458.mp4 at 30 fps...
Moved original .avi file to X:\Nimrod\pv_106_unsorted_data_04_09_25\block4\videos\avi_version\left_20250904T132458.avi
Converting X:\Nimrod\pv_106_unsorted_data_04_09_25\block4\videos\right_20250904T132458.avi to X:\Nimrod\pv_106_unsorted_data_04_09_25\block4\videos\right_20250904T132458.mp4 at 60 fps...


KeyboardInterrupt: 

In [ ]:
# This is a helper function for moving files from the preytouch analysis pipeline into another folder on sil
def block_find_preytouch_timestamp(block):
    filenames = [i.name for i in (block.arena_path / 'videos').iterdir() if '.mp4' in str(i.name)]
    pattern = r"\d{8}T\d{6}"
    timestamps = [re.search(pattern, filename).group() for filename in filenames]
    unique_ts = list(set(timestamps))
    if len(unique_ts) == 1:
        
        return unique_ts[0]
    else:
        print(f'multiple times problem {unique_ts}')

def move_parquet_files(block,origin_folder):
    origin_folder = pathlib.Path(origin_folder)
    block_t = block_find_preytouch_timestamp(block)
    parquet_files_list = [i for i in origin_folder.iterdir() if vid_t[0] in i.name]
    print(parquet_files_list)
    
for block in block_collection:
    move_parquet_files(block,r'Z:\Nimrod\experiments\PV_126\unsorted_head_loc_predictions')

In [11]:
import paramiko
from scp import SCPClient
from pathlib import Path
from typing import List

def find_and_copy_remote_files(ip: str, username: str, password: str, root_folder: str, identifier: str, local_output_dir=None):
    """
    Connects to a remote machine via SSH, finds files with a specified identifier in the name,
    and optionally copies them to a local output directory after confirmation.
    
    Parameters:
        ip (str): IP address of the remote machine.
        username (str): SSH username.
        password (str): SSH password.
        root_folder (str): The root folder on the remote machine to search in.
        identifier (str): Identifier to look for in the file names.
        local_output_dir (str): Local directory to copy the found files to.
    """
    # Initialize SSH client
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    try:
        # Connect to the remote machine
        ssh.connect(ip, username=username, password=password)
        
        # Execute the find command to list files containing the identifier in their filename
        find_command = f"find {root_folder} -type f -name '*{identifier}*'"
        stdin, stdout, stderr = ssh.exec_command(find_command)
        
        # Parse the file paths
        file_paths = [line.strip() for line in stdout]
        
        # Confirmation step
        if not file_paths:
            print("No files found with the specified identifier.")
            return
        
        
        confirm = input(f"Found {len(file_paths)} file(s) with identifier '{identifier}'. \n Do you want to copy these files to the local output directory? (y/n): ").strip().lower()
        
        if confirm != 'y':
            print("Copy operation canceled.")
            return file_paths
        
        
        # Ensure the local output directory exists
        if local_output_dir is not None:
            local_output_path = Path(local_output_dir)
            local_output_path.mkdir(parents=True, exist_ok=True)
            
            # Initialize SCP client for file transfer
            with SCPClient(ssh.get_transport()) as scp:
                for file_path in file_paths:
                    # Define the local file path
                    remote_file_path = Path(file_path)
                    local_file_path = local_output_path / remote_file_path.name
                    
                    # Copy file to local output directory
                    print(f"Copying {remote_file_path} to {local_file_path}")
                    scp.get(remote_file_path.as_posix(), local_file_path.as_posix())
    
            print("File transfer completed.")
            
    finally:
        # Close the SSH connection
        ssh.close()
        return file_paths

# Example usage:
files = find_and_copy_remote_files('132.66.42.233', 'ep-arena', 'rept!leC0rtex', '/data/PreyTouch/output/experiments/PV126/', '.parquet', local_output_dir=r'Z:\Nimrod\experiments\PV_126\unsorted_head_loc_predictions')
print(files)


Copying \data\PreyTouch\output\experiments\PV126\20240731\block5\videos\predictions\front_head_ephys_resnet_101__top_20240731T164321.parquet to Z:\Nimrod\experiments\PV_126\unsorted_head_loc_predictions\front_head_ephys_resnet_101__top_20240731T164321.parquet
Copying \data\PreyTouch\output\experiments\PV126\20240731\block3\videos\predictions\front_head_ephys_resnet_101__top_20240731T142512.parquet to Z:\Nimrod\experiments\PV_126\unsorted_head_loc_predictions\front_head_ephys_resnet_101__top_20240731T142512.parquet
Copying \data\PreyTouch\output\experiments\PV126\20240731\block4\videos\predictions\front_head_ephys_resnet_101__top_20240731T155101.parquet to Z:\Nimrod\experiments\PV_126\unsorted_head_loc_predictions\front_head_ephys_resnet_101__top_20240731T155101.parquet
Copying \data\PreyTouch\output\experiments\PV126\20240731\block1\videos\predictions\front_head_ephys_resnet_101__top_20240731T105636.parquet to Z:\Nimrod\experiments\PV_126\unsorted_head_loc_predictions\front_head_ephys_

In [14]:
import pandas as pd
df = pd.read_parquet(r"Z:\Nimrod\experiments\PV_126\unsorted_head_loc_predictions\front_head_ephys_resnet_101__top_20240804T111612.parquet", engine="pyarrow")


In [6]:
def get_paths_to_record_nodes(tree_root_path, export_path=False):
    """
    This function looks for record node folders and extracts them to a .mat file that can be used with the Generate_Metadata_with_timeSeriesViewer.mlx file located at Nimrod's desktop on the MSI server
    :param tree_root_path:
    :return:
    """
    # define some helper functions
    def match_date_pattern(string):
        pattern = r'^\d{4}_\d{2}_\d{2}$'
        return bool(re.match(pattern, string))

    def match_block_pattern(string):
        pattern = r'^block_\d{3}$'
        return bool(re.match(pattern, string))

    def convert_paths_to_unix(paths, relative_to):
        unix_paths = []
        for path in paths:
            path = path.relative_to(relative_to)
            unix_path = pathlib.Path(path).as_posix()
            unix_paths.append(unix_path)
        return unix_paths

    path_list = [] # initialize a variable to collect the paths into
    date_folder_list = [i for i in tree_root_path.iterdir() if match_date_pattern(str(i.name)) and i.is_dir()] # use function to find date folder
    for date_folder in date_folder_list: # iterate over date folders
        for block in [b for b in date_folder.iterdir() if match_block_pattern(str(b.name)) and b.is_dir()]: # use function to find block folders
            oe_path = block / 'oe_files' # enter the oe_files folder
            datetime_folder_name = [i.name for i in oe_path.iterdir()][0]
            datetime_path = oe_path / datetime_folder_name
            rec_node_name = [i.name for i in datetime_path.iterdir() if 'Node' in (str(i.name))][0]
            rec_node_path = datetime_path / rec_node_name
            path_list.append(rec_node_path)
    if export_path:
            t = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
            mat_format = {'data': path_list}
            scipy.io.savemat(export_path / f'{t}_record_nodes.mat', mat_format)


get_paths_to_record_nodes(pathlib.Path(r'Z:\Nimrod\Accelerometer_calibration_folder\Measurements'), export_path=pathlib.Path(r'Z:\Nimrod\Accelerometer_calibration_folder\Measurements'))

In [4]:
def find_parsed_events_files(root_folder):
    root_path = pathlib.Path(root_folder)
    parsed_events_files = list(root_path.rglob('parsed_events.csv'))
    
    return parsed_events_files

def get_column_labels(file_path):
    df = pd.read_csv(file_path)
    return df.columns.tolist()

root_folder_path = pathlib.Path(r"Z:\Nimrod\experiments\PV_62")
pathlist = find_parsed_events_files(root_folder_path)
for p in pathlist:
    print(p.parent.parent.parent)
    cols = get_column_labels(p)
    print(cols)

KeyboardInterrupt: 

In [2]:
import re

def match_date_pattern(string):
    pattern = r'^\d{4}_\d{2}_\d{2}$'
    return bool(re.match(pattern, string))

def match_block_pattern(string):
    pattern = r'^block_\d{3}$'
    return bool(re.match(pattern, string))

def convert_paths_to_unix(paths, relative_to):
    unix_paths = []
    for path in paths:
        path = path.relative_to(relative_to)
        unix_path = pathlib.Path(path).as_posix()
        unix_paths.append(unix_path)
    return unix_paths

path_list = []
tree_root_path = pathlib.Path('Z:/Nimrod/experiments/PV_24')
date_folder_list = [i for i in tree_root_path.iterdir() if match_date_pattern(str(i.name)) and i.is_dir()]
for date_folder in date_folder_list:
    for block in [b for b in date_folder.iterdir() if match_block_pattern(str(b.name)) and b.is_dir()]:
        oe_path = block / 'oe_files'
        datetime_folder_name = [i.name for i in oe_path.iterdir()][0]
        datetime_path = oe_path / datetime_folder_name
        rec_node_name = [i.name for i in datetime_path.iterdir() if 'Node' in (str(i.name))][0]
        rec_node_path = datetime_path / rec_node_name
        path_list.append(rec_node_path)

relative_to = pathlib.Path('Z:/Nimrod/experiments')
unix_paths = convert_paths_to_unix(path_list,relative_to=relative_to)
export_path = pathlib.Path(r'Z:\Nimrod\HelperFiles\record_node_locations_pv24.mat')
mat_format = {'data': unix_paths}
scipy.io.savemat(export_path, mat_format)

In [4]:
# construction of delta/beta ratio extraction (utiliIng marks' code):
path_to_csv = pathlib.Path(r'Z:\Nimrod\Copy_of_brainState.csv')
df = pd.read_csv(path_to_csv)

unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed_columns)
columns = df.columns
animal = 'PV57'
recFormat = 'OERecording' 
species = 'Pogona'
MEA_Layout = 'layout_40_16x2_FlexLin'
defaultLFPCh = '15'
accelerometerCh = '33,34,35'
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_57')
data = []
date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    folder_list = [i for i in date_folder.iterdir()]
    for block in folder_list:
        rec_name = block.name
        if 'block' in rec_name:
            date = date_folder.name
            path_to_record_node = [i for i in block.rglob('*') if 'Record Node' in i.name][0]
            parts = path_to_record_node.parts
            new_root = pathlib.Path('/media/sil2/Data/Nimrod')
            new_path = new_root.joinpath(*parts[2:])
            data.append({'Animal': animal,
                         'recNames':rec_name,
                         'Date': date,
                         'recFormat': recFormat,
                         'Species': species,
                         'MEA_Layout': MEA_Layout,
                         'folder': pathlib.PurePath(new_path).as_posix() + '/',
                         'defaulLFPCh': defaultLFPCh,
                         'accelerometerCh':accelerometerCh})
            

c:\users\marks3\pycharmprojects\pythonproject3\venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,7,9,11,13,14,15,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
pathlib.PurePath(new_path).as_posix() + '/'


'/media/sil2/Data/Nimrod/experiments/PV_57/2024_12_01/block_013/oe_files/PV_57_hunter_2_2024-12-01_16-34-43/Record Node 102/'

In [4]:
collected_df = pd.DataFrame.from_dict(data)

new_df = pd.concat([df.dropna(),collected_df])

new_df.to_csv(r'Z:\Nimrod\pv57_xl_for_mark_analysis.csv')
new_df

,Exclude,spikes,sortedManually,videoSync,eyeVideo,Breathing,Animal,recNames,Remarks,Date,...,defaulLFPCh.1,Temp,tempMedian,Complete_recording,TempLogger_file,Temp_verifcation,video_start_time,video_end_time,AUX_data,notes
0,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_001,NaN,2024_07_11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_002,NaN,2024_07_11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_003,NaN,2024_07_11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_004,NaN,2024_07_11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_005,NaN,2024_07_11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_006,NaN,2024_07_18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_007,NaN,2024_07_18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_008,NaN,2024_07_18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_009,NaN,2024_07_18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,PV62,block_010,NaN,2024_07_18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
meta_files = []
for p in path_list:
    m_file = [i for i in p.iterdir() if i.name == 'OE_metaData.mat']
    meta_files.append(m_file)
print(meta_files)

[[WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_005/oe_files/2023-04-22_14-30-15/Record Node 108/OE_metaData.mat')], [WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_006/oe_files/2023-04-22_14-34-56/Record Node 108/OE_metaData.mat')], [WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_007/oe_files/2023-04-22_14-41-49/Record Node 108/OE_metaData.mat')], [WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_008/oe_files/2023-04-22_14-57-30/Record Node 108/OE_metaData.mat')], [WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_009/oe_files/2023-04-22_15-14-56/Record Node 108/OE_metaData.mat')], [WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_24/block_010/oe_files/2023-04-24_16-49-28/Record Node 108/OE_metaData.mat')], [WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_24/block_011/oe_files/2023-04-24_17-09-35/Record Node 108/OE_metaData.mat')], [WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_24/block_012/oe_files/2023-04-24_17-27-10/Record Node

In [7]:
def convert_paths_to_unix(paths, relative_to):
    unix_paths = []
    for path in paths:
        path = path.relative_to(relative_to)
        unix_path = pathlib.Path(path).as_posix()
        unix_paths.append(unix_path)
    return unix_paths

In [9]:
import time
# cell to find the OE nodes in a folder tree:
relative_to = pathlib.Path(r'Z:\Nimrod\Accelerometer_calibration_folder\Measurements')
export_path = relative_to 
paths_to_record_nodes = [i for i in relative_to.rglob('*') if 'Record Node' in i.name]
# create the relative paths from mother folder:
unix_paths = convert_paths_to_unix(paths_to_record_nodes,relative_to=relative_to)

t = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
mat_format = {'data': unix_paths}
scipy.io.savemat(export_path / f'{t}_record_nodes.mat', mat_format)



In [9]:
unix_paths

['2024_11_21/block_001/oe_files/pv_57_trial01_2024-11-21_15-32-31/Record Node 102',
 '2024_11_21/block_002/oe_files/pv_57_trial02_2024-11-21_15-55-01/Record Node 102',
 '2024_11_21/block_003/oe_files/pv_57_trial03_2024-11-21_16-20-30/Record Node 102',
 '2024_11_21/block_004/oe_files/pv_57_trial04_2024-11-21_16-33-37/Record Node 102',
 '2024_11_25/block_005/oe_files/pv_57_day2_01_2024-11-25_14-59-12/Record Node 102',
 '2024_11_25/block_006/oe_files/pv_57_day2_02_2024-11-25_15-12-05/Record Node 102',
 '2024_11_25/block_007/oe_files/pv_57_day2_03_2024-11-25_15-28-31/Record Node 102',
 '2024_11_25/block_008/oe_files/pv_57_day2_05_2024-11-25_16-07-18/Record Node 102',
 '2024_11_25/block_009/oe_files/pv_57_day2_06_2024-11-25_16-25-35/Record Node 102',
 '2024_12_01/block_010/oe_files/PV_57_hunter__2024-12-01_15-37-36/Record Node 102',
 '2024_12_01/block_011/oe_files/PV_57_hunter_2_2024-12-01_15-43-28/Record Node 102',
 '2024_12_01/block_012/oe_files/PV_57_hunter_2_2024-12-01_16-08-39/Record N

In [29]:
import time
# cell to find the arena_videos in a folder tree:
relative_to = pathlib.Path(r'Z:\Nimrod\experiments\PV_57')
export_path = relative_to / 'HelperFiles'
paths_to_all_vids = [i for i in relative_to.rglob('*') if 'mp4' in i.name]
paths_to_back_arena_vids = [i for i in paths_to_all_vids if 'arena_videos' == i.parent.parent.name and 'back' in str(i.name)]
# create the relative paths from mother folder:
unix_paths = convert_paths_to_unix(paths_to_back_arena_vids, relative_to=relative_to)

t = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
mat_format = {'data': unix_paths}
scipy.io.savemat(export_path / f'{t}_arena_video_paths.mat', mat_format)

In [32]:
unix_paths

['2023_04_22/block_005/oe_files/2023-04-22_14-30-15/Record Node 108',
 '2023_04_22/block_006/oe_files/2023-04-22_14-34-56/Record Node 108',
 '2023_04_22/block_007/oe_files/2023-04-22_14-41-49/Record Node 108',
 '2023_04_22/block_008/oe_files/2023-04-22_14-57-30/Record Node 108',
 '2023_04_22/block_009/oe_files/2023-04-22_15-14-56/Record Node 108',
 '2023_04_24/block_010/oe_files/2023-04-24_16-49-28/Record Node 108',
 '2023_04_24/block_011/oe_files/2023-04-24_17-09-35/Record Node 108',
 '2023_04_24/block_012/oe_files/2023-04-24_17-27-10/Record Node 108',
 '2023_04_24/block_013/oe_files/2023-04-24_17-47-56/Record Node 108',
 '2023_04_24/block_014/oe_files/2023-04-24_18-24-02/Record Node 108',
 '2023_04_24/block_015/oe_files/2023-04-24_18-50-43/Record Node 108',
 '2023_04_25/block_016/oe_files/2023-04-25_10-17-51/Record Node 108',
 '2023_04_25/block_017/oe_files/2023-04-25_10-34-24/Record Node 108',
 '2023_04_25/block_018/oe_files/2023-04-25_11-02-43/Record Node 108',
 '2023_04_25/block_0

In [12]:

relative_to = pathlib.Path('Z:/Nimrod/experiments')
unix_paths = convert_paths_to_unix(paths_to_record_nodes,relative_to=relative_to)

export_path = pathlib.Path(r'Z:\Nimrod\HelperFiles\record_node_locations_pv57.mat')
mat_format = {'data': unix_paths}
scipy.io.savemat(export_path, mat_format)

EYE tracking video conversion and labeling from here:

In [4]:
# this code converts h264 files to mp4 in a given folder and its subfolders
def convert_to_mp4(folder_path,input_format='.h264'):
    folder_path = pathlib.Path(folder_path)
    input_format = f'.{input_format.lstrip(".")}'  # Ensure input format starts with a dot

    files_to_convert = folder_path.rglob(f'*{input_format}')
    for f in files_to_convert:
        # Check if the directory contains any .mp4 files
        parent_folder = f.parent
        mp4_files_in_folder = list(parent_folder.glob('*.mp4'))
        
        if mp4_files_in_folder:
            print(f'Skipping folder {parent_folder} because it contains .mp4 files')
            continue

        print(f'Converting {f} to mp4')
        fps_str = str(f)
        hz_index = fps_str.find('hz')
        if hz_index > 1:
            fps = fps_str[hz_index - 2:hz_index]
        else:
            fps = 60
            print('Could not determine FPS, using 60...')
        
        output_file = f.with_suffix('.mp4')
        try:
            sp.run(['MP4Box', '-fps', str(fps), '-add', str(f), str(output_file)], check=True)
            print('Done')
        except sp.CalledProcessError as e:
            print(f'Error converting {f}: {e}')

folder_path = pathlib.Path(r'X:\Nimrod\experiments\PV_106\2025_08_06')
input_format = '.h264'
convert_to_mp4(folder_path,input_format)

Converting X:\Nimrod\experiments\PV_106\2025_08_06\block_008\eye_videos\LE\pv_106_d3_t1\pv_106_d3_t1.h264 to mp4
Could not determine FPS, using 60...
Done
Converting X:\Nimrod\experiments\PV_106\2025_08_06\block_008\eye_videos\RE\pv_106_d3_t1\pv_106_d3_t1.h264 to mp4
Could not determine FPS, using 60...
Done
Converting X:\Nimrod\experiments\PV_106\2025_08_06\block_009\eye_videos\LE\pv_106_d3_t2\pv_106_d3_t2.h264 to mp4
Could not determine FPS, using 60...
Done
Converting X:\Nimrod\experiments\PV_106\2025_08_06\block_009\eye_videos\RE\pv_106_d3_t2\pv_106_d3_t2.h264 to mp4
Could not determine FPS, using 60...
Done
Converting X:\Nimrod\experiments\PV_106\2025_08_06\block_010\eye_videos\LE\pv_106_d3_t3\pv_106_d3_t3.h264 to mp4
Could not determine FPS, using 60...
Done
Converting X:\Nimrod\experiments\PV_106\2025_08_06\block_010\eye_videos\RE\pv_106_d3_t3\pv_106_d3_t3.h264 to mp4
Could not determine FPS, using 60...
Done
Converting X:\Nimrod\experiments\PV_106\2025_08_06\block_011\eye_video

In [5]:
import pathlib
import subprocess as sp
import json

def get_fps(file_path):
    try:
        result = sp.run(['ffmpeg', '-i', str(file_path), '-v', 'quiet', '-print_format', 'json', '-show_streams', '-select_streams', 'v:0'],
                        capture_output=True, text=True, check=True)
        streams = json.loads(result.stdout)['streams']
        for stream in streams:
            if stream['codec_type'] == 'video':
                return float(stream['avg_frame_rate'].split('/')[0]) / float(stream['avg_frame_rate'].split('/')[1])
    except Exception as e:
        print(f"Error getting FPS for {file_path}: {e}")
        return None

def convert_h264_to_mp4(folder_path, manual_fps):
    folder_path = pathlib.Path(folder_path)
    input_format = '.h264'
    
    # Recursively find all .h264 files in the folder and its subfolders
    files_to_convert = folder_path.rglob(f'*{input_format}')
    for f in files_to_convert:
        print(f'Converting {f} to MP4')
        if manual_fps is None:
            fps = get_fps(f)
            if fps is None:
                print(f'Skipping {f} due to FPS extraction error.')
                continue
        else:
            fps = manual_fps
        output_file = f.with_suffix('.mp4')
        try:
            sp.run(['MP4Box', '-fps', str(fps), '-add', str(f), str(output_file)], check=True)
            print('Done')
        except sp.CalledProcessError as e:
            print(f'Error converting {f}: {e}')


In [7]:
folder_path = pathlib.Path(r'X:\Nimrod\face_accelerometer_data_d1\right_eye_pi')
convert_h264_to_mp4(folder_path,60)

Converting X:\Nimrod\face_accelerometer_data_d1\right_eye_pi\pv_88_d1_fittrial_640x480_60hz_experiment_1_recording_0\pv_88_d1_fittrial.h264 to MP4
Done
Converting X:\Nimrod\face_accelerometer_data_d1\right_eye_pi\pv_88_d1_t0_640x480_60hz_experiment_1_recording_0\pv_88_d1_t0.h264 to MP4
Done
Converting X:\Nimrod\face_accelerometer_data_d1\right_eye_pi\pv_88_d1_t1_640x480_60hz_experiment_1_recording_0\pv_88_d1_t1.h264 to MP4
Done
Converting X:\Nimrod\face_accelerometer_data_d1\right_eye_pi\pv_88_d1_t2_640x480_60hz_experiment_1_recording_0\pv_88_d1_t2.h264 to MP4
Done
Converting X:\Nimrod\face_accelerometer_data_d1\right_eye_pi\pv_88_d1_t3_640x480_60hz_experiment_1_recording_0\pv_88_d1_t3.h264 to MP4
Done
Converting X:\Nimrod\face_accelerometer_data_d1\right_eye_pi\pv_88_d1_t4_640x480_60hz_experiment_1_recording_0\pv_88_d1_t4.h264 to MP4
Done
Converting X:\Nimrod\face_accelerometer_data_d1\right_eye_pi\pv_88_d1_t5_640x480_60hz_experiment_1_recording_0\pv_88_d1_t5.h264 to MP4
Done
Converti

In [14]:

folder_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_126\Unsorted_eye_vids\right_eye_pi')
input_format = '.h264'

files_to_convert = folder_path.rglob(f'*{input_format}')
for f in files_to_convert:
    print(f'converting {f} to mp4')
    fps = str(f)[str(f).find('hz') - 2:str(f).find('hz')]
    if len(fps) != 2:
        fps = 60
        print('could not determine fps, using 60...')
    
    output_file = f.with_suffix('.mp4')
    try:
        sp.run(['MP4Box', '-fps', str(fps), '-add', str(f), str(output_file)], check=True)
        print('Done')
    except sp.CalledProcessError as e:
        print(f'Error converting {f}: {e}')
        

converting Z:\Nimrod\experiments\PV_126\Unsorted_eye_vids\right_eye_pi\blank_trial1_640x480_60hz_experiment_1_recording_0\blank_trial1.h264 to mp4
Done
converting Z:\Nimrod\experiments\PV_126\Unsorted_eye_vids\right_eye_pi\blank_trial2_640x480_60hz_experiment_1_recording_0\blank_trial2.h264 to mp4
Done
converting Z:\Nimrod\experiments\PV_126\Unsorted_eye_vids\right_eye_pi\blank_trial3_640x480_60hz_experiment_1_recording_0\blank_trial3.h264 to mp4
Done
converting Z:\Nimrod\experiments\PV_126\Unsorted_eye_vids\right_eye_pi\hunter1_640x480_60hz_experiment_1_recording_0\hunter1.h264 to mp4
Done
converting Z:\Nimrod\experiments\PV_126\Unsorted_eye_vids\right_eye_pi\hunter2_640x480_60hz_experiment_1_recording_0\hunter2.h264 to mp4
Done
converting Z:\Nimrod\experiments\PV_126\Unsorted_eye_vids\right_eye_pi\IRTrial_640x480_60hz_experiment_1_recording_0\IRTrial.h264 to mp4
Done
converting Z:\Nimrod\experiments\PV_126\Unsorted_eye_vids\right_eye_pi\lenstester1_640x480_60hz_experiment_1_recording

In [16]:
# This one relabeles Left eye videos:
def label_files_in_folder(folder_path, suffix, label):
    folder_path = pathlib.Path(folder_path)
    suffix = f'.{suffix.lstrip(".")}'  # Ensure suffix starts with a dot
    files_to_label = folder_path.rglob(f'*{suffix}')
    
    for file_path in files_to_label:
        if file_path.stem.endswith(label):
            print(f'Skipping {file_path} because it is already labeled.')
            continue
        
        new_file_name = file_path.stem + label + suffix
        new_file_path = file_path.with_name(new_file_name)
        print(f'Renaming {file_path} to {new_file_path}')
        file_path.rename(new_file_path)

# Usage example:
folder_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25')
suffix = '.mp4'
label = '_RE'
label_files_in_folder(folder_path, suffix, label)

Renaming Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25\d3__01_640x480_60hz_experiment_1_recording_0\d3__01.mp4 to Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25\d3__01_640x480_60hz_experiment_1_recording_0\d3__01_RE.mp4
Renaming Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25\d3__02_640x480_60hz_experiment_1_recording_0\d3__02.mp4 to Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25\d3__02_640x480_60hz_experiment_1_recording_0\d3__02_RE.mp4
Renaming Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25\d3__03_640x480_60hz_experiment_1_recording_0\d3__03.mp4 to Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25\d3__03_640x480_60hz_experiment_1_recording_0\d3__03_RE.mp4
Renaming Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4 to Z:\Nimrod\experiments\PV_57\2024_12_01\Unsorted\pi25\d3__04_640x480_60hz_experiment_1_recording_0\d3__04_RE.mp4


In [17]:
# collect list for DLC annotation:

suffix = '.mp4'
folder_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_57')
suffix = f'.{suffix.lstrip(".")}'  # Ensure suffix starts with a dot
files_to_label = [str(f) for f in folder_path.rglob(f'*{suffix}') if 'RE' in str(f) or 'LE' in str(f) and 'labeled' not in str(f)]
files_to_label

['Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\Unsorted\\pi102\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01_LE.mp4',
 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\Unsorted\\pi102\\pv_57_single_print_02_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_02_LE.mp4',
 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\Unsorted\\pi102\\pv_57_single_print_03_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_03_LE.mp4',
 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\Unsorted\\pi102\\pv_57_single_print_04_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_04_LE.mp4',
 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\Unsorted\\pi102\\pv_57_single_print_fitting_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_fitting_LE.mp4',
 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\Unsorted\\pi25\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01_RE.mp4',
 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\Unsorted\\pi25\

Handle audio files from here:

In [11]:
# This code converts audio files in a given folder from m4a to mp3:
folder_path = pathlib.Path(r'D:/Marks3/Music/RawAudio')
input_format = 'm4a'
files_in_folder = [i for i in folder_path.iterdir()]
files_to_convert = [i for i in files_in_folder if str(input_format) in i.name]
for f in files_to_convert:
    print(f'converting {f} to mp3')
    sp.run(f'ffmpeg -i {f} -c:a libmp3lame -q:a 2 {str(f)[:-3] + "mp3"}')
    print('done')

converting D:\Marks3\Music\RawAudio\505174__kbriber__lizard-eating.m4a to mp3


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [15]:
folder_path = pathlib.Path(r'D:/Marks3/Music/RawAudio')
input_format = 'm4a'
files_in_folder = [i for i in folder_path.iterdir()]
files_to_convert = [i for i in files_in_folder if str(input_format) in i.name]
for f in files_to_convert:
    yn = input(f' will try converting {f} to mp3 - proceed?')
    if yn == 'y':
        print(f'ffmpeg -i "{str(f)} -c:a libmp3lame -q:a 2 {str(f)[:-3] + "mp3"}')
    else:
        print('not converting, next')
print('done')

not converting, next
not converting, next
ffmpeg -i D:\Marks3\Music\RawAudio\Door inside.m4a -c:a libmp3lame -q:a 2 D:\Marks3\Music\RawAudio\Door inside.mp3
ffmpeg -i D:\Marks3\Music\RawAudio\Door outside.m4a -c:a libmp3lame -q:a 2 D:\Marks3\Music\RawAudio\Door outside.mp3
ffmpeg -i D:\Marks3\Music\RawAudio\Feedersound.m4a -c:a libmp3lame -q:a 2 D:\Marks3\Music\RawAudio\Feedersound.mp3
ffmpeg -i D:\Marks3\Music\RawAudio\Natural sounds _ glass door.m4a -c:a libmp3lame -q:a 2 D:\Marks3\Music\RawAudio\Natural sounds _ glass door.mp3
done


In [9]:
str(files_to_convert[0])[:-3] + "mp3"

'D:\\Marks3\\Music\\RawAudio\\505174__kbriber__lizard-eating.mp3'

In [100]:
def move_dlc_analysis_files(directory_path, substring='DLC'):
    directory_path = pathlib.Path(directory_path)
    files = directory_path.rglob('*')
    new_dir = directory_path / "prev_analysis"
    try:
        new_dir.mkdir()
        print(f'made {new_dir}')
    except FileExistsError:
        print('prev_analysis already exists')

    for file in files:
        if file.is_file() and substring in file.name:
            new_path = new_dir / file.name
            print('file is ', file)
            print('moving to', new_path)
            shutil.move(file,new_path)
            print(f'moved {file.name} to {new_path}')






In [101]:
# search and destroy dlc files from previous annotation iterations:
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    block_list = [i for i in date_folder.iterdir()]
    for block in block_list:
        directory_LE = block / 'eye_videos' / 'LE'
        print([i for i in directory_LE.iterdir()][0])
        internal_dir = directory_LE / [i for i in directory_LE.iterdir()][0]
        move_dlc_analysis_files(directory_path=internal_dir)
        directory_RE = block / 'eye_videos' / 'RE'
        print([i for i in directory_RE.iterdir()][0])
        internal_dir = directory_RE / [i for i in directory_RE.iterdir()][0]
        move_dlc_analysis_files(directory_path=internal_dir)




Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0
prev_analysis already exists
file is  Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0\prev_analysis\pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000.csv
moving to Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0\prev_analysis\pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000.csv
moved pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000.csv to Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0\prev_analysis\pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000.csv
file is  Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0\prev_analysi

In [99]:
#clean prev_analysis folders in wrong place
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
dest = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62\UnSorted\errors')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    block_list = [i for i in date_folder.iterdir()]
    for block in block_list:
        directory_LE = block / 'eye_videos' / 'LE'
        folders = [i for i in directory_LE.glob('*')]
        for folder in folders:
            if folder.name == 'prev_analysis':
                eye = 'LE'
                # print(f'would have moved {folder} to {dest / block.name / eye / folder.name}')
                shutil.move(folder, dest / block.name / eye / folder.name)
        directory_RE = block / 'eye_videos' / 'RE'
        folders = [i for i in directory_RE.glob('*')]
        for folder in folders:
            if folder.name == 'prev_analysis':
                eye = 'RE'
                # print(f'would have moved {folder} to {dest / block.name / eye / folder.name}')
                shutil.move(folder, dest / block.name / eye / folder.name)



In [104]:
# clean empty prev_analysis folders:
def is_directory_empty(directory_path):
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file() or entry.is_dir():
                return False
    return True

p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
dest = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62\UnSorted\errors')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    block_list = [i for i in date_folder.iterdir()]
    for block in block_list:
        directory_LE = block / 'eye_videos' / 'LE'
        internal_dir = [i for i in directory_LE.glob('*')][0]
        folders = [i for i in internal_dir.glob('*')]
        for folder in folders:
            if folder.name == 'prev_analysis' and is_directory_empty(folder):
                eye = 'LE'
                #print(f'would have moved {folder} to {dest / block.name / eye / folder.name}')
                shutil.move(folder, dest / block.name / eye / folder.name)
        directory_RE = block / 'eye_videos' / 'RE'
        internal_dir = [i for i in directory_RE.glob('*')][0]
        folders = [i for i in internal_dir.glob('*')]
        for folder in folders:
            if folder.name == 'prev_analysis' and is_directory_empty(folder):
                eye = 'RE'
                #print(f'would have moved {folder} to {dest / block.name / eye / folder.name}')
                shutil.move(folder, dest / block.name / eye / folder.name)

AttributeError: 'WindowsPath' object has no attribute 'rstrip'

In [62]:
# correct nested prev_analysis folder:
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
directory = date_folder_list[0] / 'block_005'/ 'eye_videos' / 'LE' / 'pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0'
files = directory.rglob('*/')
for file in files:
    if 'DLC' in str(file):
        shutil.move(file,directory / file.name)


In [4]:
## This section creates a .csv based report on all experimental blocks for a single animal
df_dict = {
    'date':[],
    'block':[],
    'oe_file':[],
    'eye_vids_h264':[],
    'eye_vids_mp4':[],
    'eye_vids_labeled':[],
    'arena_vids':[],
    'notes':[]
}
df = pd.DataFrame(df_dict)

p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    block_list = [i for i in date_folder.iterdir()]
    for block in block_list:
        analysis = [i for i in (block / 'analysis').iterdir()]
        h264s = [ pathlib.Path(f).name for f in glob.glob(str(block / 'eye_videos') + r'\**\*.h264',recursive=True) if 'DLC' not in str(f)]
        mp4s = [ pathlib.Path(f).name for f in glob.glob(str(block / 'eye_videos') + r'\**\*.mp4',recursive=True) if 'DLC' not in str(f)]
        labeled_videos = [ pathlib.Path(f).name for f in glob.glob(str(block / 'eye_videos') + r'\**\*.mp4',recursive=True) if 'DLC' in str(f)]
        arena_videos = [ pathlib.Path(f).name for f in glob.glob(str(block / 'arena_videos') + r'\**\*.mp4',recursive=True)]
        try:
            oe_file = [i for i in (block / 'oe_files').iterdir()][0].name
        except IndexError:
            oe_file = None
        df_row = {
        'date':date_folder.name,
        'block':block.name[6:],
        'eye_vids_h264':h264s,
        'eye_vids_mp4':mp4s,
        'eye_vids_labeled':labeled_videos,
        'arena_vids':arena_videos,
        'oe_file': oe_file
        }
        i = len(df)
        df.loc[i] = df_row




c:\users\marks3\pycharmprojects\pythonproject3\venv\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [28]:
## This cell re-organizes date names
# find all folders to rename:
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
for i in date_folder_list:
    date = i.name
    day = date[0:2]
    month = date[3:5]
    year = date[-4:]
    new_name = year+'_'+month+'_'+day
    new_path = i.parent / new_name
    i.rename(new_path)



In [4]:
# This cell organizes block names into block_XXX format
# iterate over all date folders:
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    folder_list = [i for i in date_folder.iterdir()]
    for block in folder_list:
        # find the block number
        name = block.name
        # There should be a regex thing here to deal with different possible locations
        pattern = re.compile(r"\d+")
        matches = pattern.finditer(name)
        # correct the number to the new format
        for match in matches:
            num = match.group()
            span = match.span()
        if len(num) == 1:
            new_num = f'00{num}'
        elif len(num) == 2:
            new_num = f'0{num}'
        else:
            new_num = num
        #print(f'the number {num} became {new_num}')
        new_block_name = f'block_{new_num}'
        new_block_path = block.parent / new_block_name
        #print(f'would have renamed {block} to {new_block_path}')
        block.rename(new_block_path)

In [114]:
# Rename AUX labeled files as CHxs
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'BAD_BLOCK' not in str(i) and (p/str(i)).is_dir() and 'template' not in str(i)]
for date_folder in date_folder_list:
    for block in (p / date_folder).iterdir():
        oe_folder = block / 'oe_files'
        oe_folder_name = [i for i in oe_folder.iterdir() if i.is_dir]
        try:
            oe_folder = oe_folder_name[0] / r'Record Node 101'
        except IndexError:
            print('no open ephys folder for', block)
        for i in oe_folder.iterdir():
            if 'AUX' in str(i):
                ch_num = str(i)[-12]
                ch_num = str(int(ch_num) + 32)
                destination = i.parent / (str(i.name[:4]) + 'CH' + ch_num + '.continuous')
                #os.rename(i,destination)

no open ephys folder for Z:\Nimrod\experiments\PV_24\04_01_2022\block_65_BAD_BLOCK


In [113]:
# extract hidehere files back into the block:
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'BAD_BLOCK' not in str(i) and (p/str(i)).is_dir() and 'template' not in str(i)]
for date_folder in date_folder_list:
    for block in (p / date_folder).iterdir():
        oe_folder = block / 'oe_files'
        oe_folder_name = [i for i in oe_folder.iterdir() if i.is_dir]
        try:
            oe_folder = oe_folder_name[0] / r'Record Node 101'
        except IndexError:
            print('no open ephys folder for', block)
        if (oe_folder / 'hidehere').is_dir():
            print(f'take care of block {oe_folder}')
        else:
            print(f'nothing here')


nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
no open ephys folder for Z:\Nimrod\experiments\PV_24\04_01_2022\block_65_BAD_BLOCK
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing

In [65]:
df = pd.DataFrame(data=None, index=df_inds, columns=['automated_step','manual_synchronization','bokeh_plot', 'saccade_analysis', 'accel_analysis', 'db_ratio'])

for i, block in enumerate(blocklist):
    ind = df.index[i]
    # collect it:
    a_files = os.listdir(block.block_path / 'analysis')
    if 'blocksync_df.csv' in a_files:
        df.at[ind,'automated_step'] = True
    if len([f for f in a_files if '.html' in f]) != 0:
        df.at[ind,'bokeh_plot'] = True
    if 'manual_sync_df.csv' in a_files:
        df.at[ind,'manual_synchronization'] = True
    if 'l_saccades.csv' in a_files:
        df.at[ind,'saccade_analysis'] = True
    # find the accel file location:
    if (block.oe_path / 'analysis').is_dir():
        oep = block.oe_path / 'analysis'
        oe_files = glob.glob(str(oep)+'\*\*',recursive=True)

        for f in oe_files:
            if 'lizMov' in f:
                df.at[ind,'accel_analysis'] = True
            elif 'dbRatio' in f:
                df.at[ind,'db_ratio'] = True



In [79]:
# reptilearn data sorter draft
p = pathlib.Path(r"D:\experiments\Transients\reptilearn_recordings")
files = os.listdir(p)
for file in files:
        name = file[-19:]
        pre_name = file[:-19]
        format_date = name[6:8] + '_' + name[4:6] + '_' + name[0:4]
        format_time = name[9:11] + '_' + name[11:13] + '_' + name[13:15]
        origin = p / file
        destination = p / format_date
        print(f'would have moved {origin} to {destination}')
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        destination = p / format_date / format_time
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        try:
            shutil.move(origin,destination / file)
        except FileExistsError:
            print('this file is already there, moving on...')

would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_back_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_back_20220116-130043.mp4 to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_left_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_left_20220116-130043.mp4 to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_right_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Trans

In [17]:
def reptilearn_data_sorter(path_to_dir):
    """
    This function takes a bunch of reptilearn data and sorts them by date and time as per the given format: SomeTitle_CameraName_YYYYMMDD_HHMMSS.mp4 or csv and sorts them as follows:
     date_folder --> Time_folder --> files
    :param path_to_dir: Pathlib.Path object which points to the directory of choice
    :return:
    """
    p = path_to_dir
    files = os.listdir(p)
    for file in files:
        if os.path.isdir(p / file):
            continue
        name = file[-19:]
        format_date = name[6:8] + '_' + name[4:6] + '_' + name[0:4]
        format_time = name[9:11] + '_' + name[11:13] + '_' + name[13:15]
        origin = p / file
        destination = p / format_date
        #print(f'would have moved {origin} to {destination}')
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        destination = p / format_date / format_time
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        try:

            shutil.move(origin,destination / file)
        except FileExistsError:
            print('this file is already there, moving on...')

In [18]:
reptilearn_data_sorter(pathlib.Path(r'D:\experiments\Transients\reptilearn_recordings'))

destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination 

In [166]:
def reptilearn_organizer_date_to_date(date, path_to_animal_folder, path_to_arena_files):
    """
    This function takes two date folders (with the exact same date format and values) and matches the arena files to OE time signatures - This should be done after all files are organized into date folders on both sides and Open ephys files are in their correct place within the block tree
    :param date: DD_MM_YYYY
    :param path_to_animal_folder: the path to the animal folder (for example, "D:\experiments\PV_24")
    :param path_to_arena_files:  the path to the reptilearn files folder (for example "D:\experiments\Transients\reptilearn_recordings"
    :return: None
    """
    p_dst = pathlib.Path(rf'{path_to_animal_folder}\{date}') #path to a specific date in the animal folder
    p_ori = pathlib.Path(rf'{path_to_arena_files}\{date}') # path to the arena files sorted by date
    dst_ls = os.listdir(p_dst)
    ori_ls = os.listdir(p_ori)
    dst_times = []
    for file in dst_ls:
        p_oe = p_dst / file / 'oe_files'
        filename = os.listdir(p_oe)
        if len(filename) != 1:
            print(f'Missing open ephys file in {file}, cannot work automatically')
            continue
        oe_time = filename[0][-8:]
        t = datetime.timedelta(hours=int(oe_time[0:2]),
                               minutes=int(oe_time[3:5]),
                               seconds=int(oe_time[6:8]))
        dst_times.append([file,t])

    if len(ori_ls) == len(dst_ls):
        print(f'same file count')
        for file in ori_ls:
            arena_time = datetime.timedelta(hours=int(file[0:2]),
                                            minutes=int(file[3:5]),
                                            seconds=int(file[6:8]))
    #        print(f'{file} deltas are')
            deltas = []
            for i in range(len(dst_times)):
                delta = arena_time - dst_times[i][1]
                deltas.append(delta.seconds)
    #            print(f'{dst_times[i][0]} => {delta.seconds}')
            block_position = np.argmin(deltas)
            dst_block = dst_times[block_position][0]
            files_to_move = [p_ori / file / f for f in os.listdir(p_ori / file)]
            print(f'will move {arena_time} to {p_dst / dst_block}')
            print(f'because the delta is {deltas[block_position]}')
            for f in files_to_move:
                shutil.move(f,p_dst / dst_block / "arena_videos" / f.name)
                print(f'moved {f} to {p_dst / dst_block /"arena_videos"/f.name}')
    else:
        print('The file count does not match - work manually you lazy bum')
        print('okay, here are the time differences:')
        for file in ori_ls:
            arena_time = datetime.timedelta(hours=int(file[0:2]),
                                            minutes=int(file[3:5]),
                                            seconds=int(file[6:8]))
    #        print(f'{file} deltas are')
            deltas = []
            print(f'these are the deltas for {file}')
            for i in range(len(dst_times)):
                delta = arena_time - dst_times[i][1]
                deltas.append(delta.seconds)
                print(f'{dst_times[i][0]} => {delta.seconds}')

In [167]:
reptilearn_organizer_date_to_date('04_01_2022', r"D:\experiments\PV_24", r"D:\experiments\Transients\reptilearn_recordings")

Missing open ephys file in block_65, cannot work automatically
same file count
will move 10:41:39 to D:\experiments\PV_24\04_01_2022\block_60
because the delta is 8
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\back_20220104-104139.csv to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\back_20220104-104139.csv
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\back_20220104-104139.mp4 to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\back_20220104-104139.mp4
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\left_20220104-104139.csv to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\left_20220104-104139.csv
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\left_20220104-104139.mp4 to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\left_20220104-104139.mp4
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\right_20220104-104139.csv to D:\exper

In [122]:
def build_file_tree(path, blocks_to_build, animal_name_number):
    """
    This function creates the empty file trees required to populate later
    :param path: pathlib.Path location to start the tree
    :param blocks_to_build: an iterable containing block numbers to build
    :param animal_name_number: the name of the folder where the block trees will be created
    """
    #build tree root for specific animal
    try:
        tree_root = pathlib.Path(path / animal_name_number)
        os.mkdir(tree_root)
    except FileExistsError:
        print('Animal folder already exists')
    #create a block template folder
    try:
        block_root = pathlib.Path(tree_root / 'block_template')
        os.mkdir(block_root)
        #make the block template itself:
        os.mkdir(block_root / 'analysis')
        os.mkdir(block_root / 'oe_files')
        os.mkdir(block_root / 'eye_videos')
        os.mkdir(block_root / 'eye_videos' / 'LE')
        os.mkdir(block_root / 'eye_videos' / 'RE')
        os.mkdir(block_root / 'arena_videos')
    except FileExistsError:
        print('template tree already exists')
    #duplicate the template to the number of blocks needed:
    for i in blocks_to_build:
        try:
            folder_to_create = tree_root / f'block_{i}'
            shutil.copytree(block_root,folder_to_create)
            print(f'created {folder_to_create}')
        except FileExistsError:
            print(f'block number {i} already there')

In [179]:
build_file_tree(pathlib.Path(r"D://experiments"),
                range(32,94),
                "PV_24")

Animal folder already exists
template tree already exists
block number 32 already there
block number 33 already there
block number 34 already there
block number 35 already there
block number 36 already there
block number 37 already there
block number 38 already there
block number 39 already there
block number 40 already there
block number 41 already there
block number 42 already there
block number 43 already there
block number 44 already there
block number 45 already there
block number 46 already there
block number 47 already there
block number 48 already there
block number 49 already there
block number 50 already there
block number 51 already there
block number 52 already there
block number 53 already there
block number 54 already there
block number 55 already there
block number 56 already there
block number 57 already there
block number 58 already there
block number 59 already there
block number 60 already there
block number 61 already there
block number 62 already there
block number

In [188]:
# open ephys file mover VERSION1
#initialize
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
path_to_oe_data = pathlib.Path(r'\\sil1\Data\Pogona Vitticeps\PV24')
folders_to_move = [i for i in os.listdir(path_to_oe_data) if 'EE' in i]

#move open ephys files
for folder in folders_to_move:
    s = folder[folder.find('EE'):]
    boolean_digit_locations = [b.isdigit() for b in s]
    boolean_underscore_locations = [b == '_' for b in s]
    fd_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_digit_locations) if x]
    funder_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_underscore_locations) if x]
    if fd_inds[0] < funder_inds[0]:
        block_num = folder[fd_inds[0]:funder_inds[0]]
    else:
        block_num = folder[fd_inds[0]:funder_inds[1]]
    if int(block_num) > 32:
        #print(f'folder {folder} will go to block number {block_num}')
        origin = pathlib.Path(path_to_oe_data / folder)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'oe_files' / folder)
        print(f'Copying {origin} to {destination}')
        try:
            shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE68_2022-01-05_14-40-31 to D:\experiments\PV_24\block_68\oe_files\PV24_05_01_22_EE68_2022-01-05_14-40-31
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE69_2022-01-05_15-12-37 to D:\experiments\PV_24\block_69\oe_files\PV24_05_01_22_EE69_2022-01-05_15-12-37
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE70_2022-01-05_15-48-24 to D:\experiments\PV_24\block_70\oe_files\PV24_05_01_22_EE70_2022-01-05_15-48-24
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE71_2022-01-05_16-20-18 to D:\experiments\PV_24\block_71\oe_files\PV24_05_01_22_EE71_2022-01-05_16-20-18
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE72_2022-01-05_16-44-09 to D:\experiments\PV_24\block_72\oe_files\PV24_05_01_22_EE72_2022-01-05_16-44-09
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_10_01_22_EE73_2022-01-10_13-55-31 to D:\experiments\PV_24\block_73\oe_files\PV24_10_01_22_EE73_2022-01-10_13-55-31
Copying \\sil1\Data\Pogona V

In [ ]:
# open ephys file mover VERSION2

In [126]:
# Move eye camera files __VERSION1__
# Initialize:
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
eyevids_path_L = pathlib.Path(r"D://experiments//Transients//24//RPI_L_BACKUP_19_01_22")
eyevids_path_R = pathlib.Path(r"D://experiments//Transients//24//RPI_R_BACKUP_19_01_22")
prefixlist = ['LE','RE']
for i, eye in enumerate([eyevids_path_L, eyevids_path_R]):
    folders_to_move = [i for i in os.listdir(eye) if 'EE' in i]
    prefixer = prefixlist[i]
    for s in folders_to_move:
        l = s.find('EE')
        #print(s[l+2:l+5])
        if s[l:l+2] == 'EE_':
            block_num =
        elif s[l+2] == '_':
            #print(f'the block number for {s} is {s[l+3]}')
            block_num = s[l+3]
        elif s[l+3] == '_':
            #print(f'the block number for {s} is {s[l+2]}')
            block_num = s[l+2]
        elif s[l+4] == '_':
            #print(f'the block number for {s} is {s[l+2:l+4]}')
            block_num = s[l+2:l+4]
        origin = pathlib.Path(eye / s)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'eye_videos' / prefixer / s)
        print(f'Copying {origin} to {destination}')
        try:
            #shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE33_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_33\eye_videos\LE\PV24_EE33_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE34_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_34\eye_videos\LE\PV24_EE34_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE35_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_35\eye_videos\LE\PV24_EE35_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE36_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_36\eye_videos\LE\PV24_EE36_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE37_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_37\eye_videos\LE\PV24_EE37_640x480_60hz_experiment_1_re

In [180]:
#eye video block identifier + mover Version 2

path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
eyevids_path_L = pathlib.Path(r"D://experiments//Transients//24//RPI_L_BACKUP_19_01_22")
eyevids_path_R = pathlib.Path(r"D://experiments//Transients//24//RPI_R_BACKUP_19_01_22")
prefixlist = ['LE','RE']
for i, eye in enumerate([eyevids_path_L, eyevids_path_R]):
    folders_to_move = [i for i in os.listdir(eye) if 'EE' in i]
    prefixer = prefixlist[i]
    for folder in folders_to_move:
        s = folder[folder.find('EE'):]
        boolean_digit_locations = [b.isdigit() for b in s]
        boolean_underscore_locations = [b == '_' for b in s]
        fd_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_digit_locations) if x]
        funder_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_underscore_locations) if x]
        if fd_inds[0] < funder_inds[0]:
            block_num = folder[fd_inds[0]:funder_inds[0]]
        else:
            block_num = folder[fd_inds[0]:funder_inds[1]]
        origin = pathlib.Path(eye / folder)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'eye_videos' / prefixer / folder)
        print(f'Copying {origin} to {destination}')
        try:
            shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE33_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_33\eye_videos\LE\PV24_EE33_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE34_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_34\eye_videos\LE\PV24_EE34_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE35_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_35\eye_videos\LE\PV24_EE35_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE36_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_36\eye_videos\LE\PV24_EE36_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Trans

In [84]:
#Dating the blocks:
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
block_num = 1
oe_path = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'oe_files')
s = os.listdir(oe_path)[0]
date_starts = s.find("2021")
date = s[date_starts:date_starts +10]
date


'2021-12-23'

In [108]:
#cleanup_DLC_files
def cleanup_dlc_files(block_path):
    prefixlist = ['LE','RE']
    for prefixer in prefixlist:
        path = block_path / 'eye_videos' / prefixer
        vid_folder = os.listdir(path)
        path = path / vid_folder[0]
        files_to_save = [path / i for i in os.listdir(path) if 'labeled.mp4' in i]
        for file in files_to_save:
            shutil.move(pathlib.Path(file),path / 'analysis' )
        files_to_remove = [path / i for i in os.listdir(path) if 'DLC' in i]
        for file in files_to_remove:
            os.remove(file)


In [109]:
for i in range(1,6):
    block_path = pathlib.Path(rf'D:\experiments\PV_24\23_12_2021\block_{i}')
    cleanup_dlc_files(block_path)

In [120]:
for i in range(1,6):
    block_path = pathlib.Path(rf'D:\experiments\PV_24\23_12_2021\block_{i}')
    prefixlist = ['LE','RE']
    for prefixer in prefixlist:
       path = block_path / 'eye_videos' / prefixer
       vid_folder = os.listdir(path)
       path = path / vid_folder[0]
       file_name = [i for i in os.listdir(path) if 'oldLabeled.mp4' in i][0]
       file_to_move = path / file_name
       shutil.move(file_to_move, block_path / 'analysis' / file_name)
       #os.rename(path / 'analysis',path / f'{base_name[:-4]}_oldLabeled.mp4')

In [18]:
#### build here: data comparison (check if all data from one computer is present in another for backup purposes) ####
# create a list of data files that exist on computer 1 and either compare them to an existing file or create a file for further comparison:
create_file = True
export_path = pathlib.Path(r'D:\experiments\Transients\file_comparison_for_backup.csv')
p = pathlib.Path(r'D:\experiments\PV_24')
p_ls = list(p.glob('**/oe_files/**'))
name_list = [i.name for i in p_ls if 'PV24' in i.name]
name_dict = {
    'folder_names': name_list
}
if create_file:
    df = pd.DataFrame(name_dict)
    df.to_csv(export_path)
else:
    print('did not export file')

In [32]:
####CAREFUL!!!!####
# This cell deletes all matlab analysis files of a given name
filename = 'lizMov.mat'
# collect date folders:
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
# for each date, find the block names:
for date_folder in date_folder_list:
    # list all blocks in the folder:
    folder_list = [i for i in date_folder.iterdir()]
    for block in folder_list:
        oe_folder = block / 'oe_files'
        oe_name = [i for i in oe_folder.iterdir()]
        if len(oe_name) == 1:
            try:
                path_to_analysis = oe_name[0] / 'Record Node 101' / 'analysis'
                analysis_folder = [i for i in path_to_analysis.iterdir()][0]
                if filename in [i for i in analysis_folder.iterdir()][0].name:
                    #os.remove(analysis_folder / filename)
                    print(f'would have removed {analysis_folder / filename}')
            except FileNotFoundError:
                print(f'error in {path_to_analysis}')

IndexError: list index out of range